In [33]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import weightedAverage
from torch.utils.data import DataLoader
from torch.nn import L1Loss
from torch.optim import Adam

In [28]:
model_list = !ls ../Model

In [29]:
model_list = ['../Model/'+model for model in model_list if 'npy' in model and 'y_val' not in model]

In [13]:
# train = pd.read_csv("../Data/train.csv", 
#                     dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
# train = train.values
# batch_size = 16
# normalFun = normalize_log2
# val_gen = SequenceGenSpec(train,False,normalFun=normalFun)
# val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)
# y_ = np.concatenate([y for x,y in val_gen],0)
# np.save('../Model/y_val.npy',y_)

In [108]:
train_gen = weightedAverageGen(model_list,'../Model/y_val.npy')
train_gen = DataLoader(train_gen,16,True,num_workers=2)

In [109]:
model = weightedAverage(5).to('cuda:0')

In [110]:
loss_func_nor = loss_func_generator(L1Loss())
opt = Adam(trainable_parameter(model),lr=1e-4)

In [111]:
model = fit(20, model, loss_func_nor, opt, train_gen)

epoch:0, train_loss:2.3643425860494935
epoch:1, train_loss:2.3619087799540104
epoch:2, train_loss:2.362142261469139
epoch:3, train_loss:2.3604611653201983
epoch:4, train_loss:2.3608362067420527
epoch:5, train_loss:2.3573954757654443
epoch:6, train_loss:2.3625434232207962
epoch:7, train_loss:2.361008513648555
epoch:8, train_loss:2.3603976897473604
epoch:9, train_loss:2.35554468181898
epoch:10, train_loss:2.362026327061203
epoch:11, train_loss:2.357865848631229
epoch:12, train_loss:2.3678984844459676
epoch:13, train_loss:2.3619265106489076
epoch:14, train_loss:2.3558560632309824
epoch:15, train_loss:2.355962098769422
epoch:16, train_loss:2.3597485951657564
epoch:17, train_loss:2.359714130185685
epoch:18, train_loss:2.3612887252051875
epoch:19, train_loss:2.359724415923065
Training completed in 4.295798301696777s


In [91]:
yhat = np.stack([np.load(model) for model in model_list],1)

In [ ]:
y_ = np.load('../Model/y_val.npy')

Median is best

In [95]:
np.mean(np.abs(y_-yhat.mean(1)))

2.3194153

In [97]:
np.mean(np.abs(y_-np.median(yhat,1)))

2.3108249

In [112]:
np.mean(np.abs(y_-yhat@(model.get_weight().cpu().detach().numpy())))

2.3172023